# 모듈 불러오기

In [1]:
# default module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import random

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [3]:
#Our Model
#Tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb

#Regression
from sklearn.linear_model import LogisticRegression

#Svm
from sklearn.svm import SVC

##### CUSTOM FUNCTION

In [4]:
def select_important_feautre(columns, feature_importance, num):
    
    #딕셔너리
    dic = {}
    for value in [list(v) for v in zip(columns, feature_importance)]:
        dic[value[1]] = value[0] 
        
    sorted_list = sorted(dic)
    sorted_list.reverse()
    
    i = 0
    result = []
    for y in sorted_list:
        result.append(dic[y])
        i = i+1
        if(i == num):
            break
    
    return result

In [5]:
def cross_val_score_custom(model, x, y, cv, dev = 0):
    
    print('\n\n')
    
    result = []
    
    for train_idx, test_idx in cv.split(x):
        
        new_x_train = x.iloc[train_idx, :]
        new_y_train = y.iloc[train_idx]
        
        new_x_test = x.iloc[test_idx, :]
        new_y_test = y.iloc[test_idx]
        
        model.fit(new_x_train, new_y_train)
        
        temp = np.mean(f1_score(new_y_test, model.predict(new_x_test), average=None))
        
        if dev:
            print('dev Mode # : {}'.format(temp))
        
        result.append(temp)
    
    print('\n\n')
    
    return np.mean(result)

In [6]:
def get_oof(model, x_train, y_train, x_test):
    
    le = LabelEncoder()
    le.fit(y_train)
    y_train = pd.Series(le.transform(y_train))
    
    oof_train = np.zeros((x_train_load_fin_v2.shape[0],))
    oof_test = np.zeros((x_test_load_fin_v2.shape[0],))
    oof_test_skf = np.empty((5, x_test_load_fin_v2.shape[0]))
    
    for i, (train_idx, test_idx) in enumerate(KFold(n_splits=5).split(x_train)):
        
        x_tr = x_train.iloc[train_idx, :]
        y_tr = y_train.iloc[train_idx]
        x_te = x_train.iloc[test_idx, :]
        
        model.fit(x_tr, y_tr)

        oof_train[test_idx] = model.predict(x_te)
        oof_test_skf[i, :] = model.predict(x_test)
    
    # version 1
    # oof_test[:] = np.round(oof_test_skf.mean(axis=0))
    
    # version 2
    oof_test[:] = pd.DataFrame(oof_test_skf).apply(lambda v: v.value_counts().idxmax(), axis=0)
    
    oof_train = [int(v) for v in oof_train]
    oof_train = le.inverse_transform(oof_train)
    
    oof_test = [int(v) for v in oof_test]
    oof_test = le.inverse_transform(oof_test)
    
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)

## 데이터셋

##### 트레인셋
- train_activity_final_v2 (675, 656, 588, 583, 560, 519, 518, 467)
- train_data (738, 744, 728, 700, 657, 654, 636, 482)
- train_activity_296
---
##### 테스트셋
- test_activity_final_v2 (675, 656, 588, 583, 560, 519, 518, 467)
- test_data (738, 744, 728, 700, 657, 654, 636, 482)
- test_activity_296

In [7]:
#트레인셋 - 로드
x_train_load_fin_v2 = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_activity_final_v2.csv').sort_values(by='acc_id', ascending=True)
x_train_load_main = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_data.csv').sort_values(by='acc_id', ascending=True)
x_train_load_296 = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_activity_296.csv').sort_values(by='acc_id', ascending=True)
y_train_load = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_label.csv').sort_values(by='acc_id', ascending=True)

In [8]:
#테스트셋 - 로드
x_test_load_fin_v2 = pd.read_csv('C://Users//zeus_//Desktop//champion_data//test/test_activity_final_v2.csv').sort_values(by='acc_id', ascending=True)
x_test_load_main = pd.read_csv('C://Users//zeus_//Desktop//champion_data//test/test_data.csv').sort_values(by='acc_id', ascending=True)
x_test_load_296 = pd.read_csv('C://Users//zeus_//Desktop//champion_data//test/test_activity_296.csv').sort_values(by='acc_id', ascending=True)

#### 공통피처만들기
---

In [19]:
#x_train_fin
x_train_fin_default = x_train_load_fin_v2.drop(['acc_id'], axis=1)
x_test_fin_default = x_test_load_fin_v2.drop(['acc_id'], axis=1)

#x_train_main
x_train_main_default = x_train_load_main.drop(['acc_id'], axis=1)
x_test_main_default = x_test_load_main.drop(['acc_id'], axis=1)

#x_train_296
x_train_296 = x_train_load_296.drop(['acc_id'], axis=1)
x_test_296 = x_test_load_296.drop(['acc_id'], axis=1)

#y_train
y_data = y_train_load.label

- fin 피처

In [20]:
feature_fin = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=30)
feature_fin.fit(x_train_fin_default, y_data)
feature_fin = select_important_feautre(x_train_fin_default.columns, feature_fin.feature_importances_, 677)

- main 피처

In [21]:
feature_main = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=30)
feature_main.fit(x_train_main_default, y_data)
feature_main = select_important_feautre(x_train_main_default.columns, feature_main.feature_importances_, 754)

### 트레인셋
---
- fin train set

In [22]:
x_train_fin_675 = x_train_fin_default.loc[:, feature_fin[0:675]]
x_train_fin_656 = x_train_fin_default.loc[:, feature_fin[0:656]]
x_train_fin_588 = x_train_fin_default.loc[:, feature_fin[0:588]]
x_train_fin_583 = x_train_fin_default.loc[:, feature_fin[0:583]]
x_train_fin_560 = x_train_fin_default.loc[:, feature_fin[0:560]]
x_train_fin_519 = x_train_fin_default.loc[:, feature_fin[0:519]]
x_train_fin_518 = x_train_fin_default.loc[:, feature_fin[0:518]]
x_train_fin_467 = x_train_fin_default.loc[:, feature_fin[0:467]]

- main train set

In [23]:
x_train_main_738 = x_train_main_default.loc[:, feature_main[0:738]]
x_train_main_744 = x_train_main_default.loc[:, feature_main[0:744]]
x_train_main_728 = x_train_main_default.loc[:, feature_main[0:728]]
x_train_main_700 = x_train_main_default.loc[:, feature_main[0:700]]
x_train_main_657 = x_train_main_default.loc[:, feature_main[0:657]]
x_train_main_654 = x_train_main_default.loc[:, feature_main[0:654]]
x_train_main_636 = x_train_main_default.loc[:, feature_main[0:636]]
x_train_main_482 = x_train_main_default.loc[:, feature_main[0:482]]

### 테스트셋
---
- fin test set

In [24]:
x_test_fin_675 = x_test_fin_default.loc[:, feature_fin[0:675]]
x_test_fin_656 = x_test_fin_default.loc[:, feature_fin[0:656]]
x_test_fin_588 = x_test_fin_default.loc[:, feature_fin[0:588]]
x_test_fin_583 = x_test_fin_default.loc[:, feature_fin[0:583]]
x_test_fin_560 = x_test_fin_default.loc[:, feature_fin[0:560]]
x_test_fin_519 = x_test_fin_default.loc[:, feature_fin[0:519]]
x_test_fin_518 = x_test_fin_default.loc[:, feature_fin[0:518]]
x_test_fin_467 = x_test_fin_default.loc[:, feature_fin[0:467]]

- main test set

In [25]:
x_test_main_738 = x_test_main_default.loc[:, feature_main[0:738]]
x_test_main_744 = x_test_main_default.loc[:, feature_main[0:744]]
x_test_main_728 = x_test_main_default.loc[:, feature_main[0:728]]
x_test_main_700 = x_test_main_default.loc[:, feature_main[0:700]]
x_test_main_657 = x_test_main_default.loc[:, feature_main[0:657]]
x_test_main_654 = x_test_main_default.loc[:, feature_main[0:654]]
x_test_main_636 = x_test_main_default.loc[:, feature_main[0:636]]
x_test_main_482 = x_test_main_default.loc[:, feature_main[0:482]]

## 모델 - Level 1 & Level 2 Stacking
---
##### Level 1
- RF, ET, GB, LGB, XGB
- LLR, SMLR
- SVM

##### Level 2
- RF or XGB

In [26]:
#랜덤포레스트 16개
rf_f1 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_f2 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_f3 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_f4 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_f5 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_f6 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_f7 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_f8 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_m1 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_m2 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_m3 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_m4 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_m5 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_m6 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_m7 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)
rf_m8 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=28, n_jobs=-1)

In [27]:
#랜덤포레스트 2개
rf_f = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=30, n_jobs=-1)
rf_m = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=30, n_jobs=-1)

In [28]:
#엑스트라트리 2개
et_f = ExtraTreesClassifier(n_estimators=100, max_depth=20)
et_m = ExtraTreesClassifier(n_estimators=100, max_depth=20)

In [29]:
#그래디언트부팅팅 2개
#GradientBoostingClassifier(learning_rate=0.1, max_depth=8, max_features=0.3, min_samples_leaf=20,n_estimators=50)
gb_f = GradientBoostingClassifier(max_depth=1, learning_rate=0.1, max_features=30, n_estimators=100)
gb_m = GradientBoostingClassifier(max_depth=1, learning_rate=0.1, max_features=30, n_estimators=100)

In [30]:
#XGB 2개
xgb_f = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.5, learning_rate=0.02,
       max_delta_step=0,min_child_weight=20, max_depth=20,
       missing=None,
       n_estimators=1, n_jobs=-1, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)

xgb_m = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.5, learning_rate=0.02,
       max_delta_step=0, max_depth=20, min_child_weight=20, missing=None,
       n_estimators=1, n_jobs=-1, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)

In [214]:
#LGB 2개

data_f = lgb.Dataset(x_train_fin_default, label= le.transform(y_data),
                            free_raw_data=False,
                            feature_name=list(x_train_fin_default.columns),
                            categorical_feature='auto')

#grid탐색에서 나온 결과로 모델을 만들어 넣어준다.
params={"boosting_type":'gbdt', "colsample_bytree":0.65,
        "learning_rate":0.1,"max_bin":700, "n_estimators":100,
        "num_leaves": 100, "objective":'multiclass',
        "random_state": 501, "reg_alpha":1, "reg_lambda": 1,
        "subsample":0.75, "max_depth": -1, "nthread": 3,
        "subsample_for_bim":200, "subsample_freq":1,"min_split_gain":0.5,
        "min_child_weight":1, "min_child_samples":5, 
        "scale_pos_weight":1, "num_class":4,'metric' : 'multi_logloss'}

gbm= lgb.train(params, data_f,100000)  #일종의 fit하는 역할

c:\python\venv\data_science\lib\site-packages\lightgbm\engine.py:102: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [31]:
#로지스틱 회귀, 소프트맥스 회귀
llr = LogisticRegression()
smlr = LogisticRegression(multi_class="multinomial", solver="lbfgs", C = 10)

In [32]:
#서포트벡터머신
# svm = SVC(kernel='rbf', C=10, gamma=1)

## 학습

In [33]:
oof_train_rf_f1, oof_test_rf_f1 = get_oof(rf_f1, x_train_fin_675, y_data, x_test_fin_675)
oof_train_rf_f2, oof_test_rf_f2 = get_oof(rf_f2, x_train_fin_656, y_data, x_test_fin_656)
oof_train_rf_f3, oof_test_rf_f3 = get_oof(rf_f3, x_train_fin_588, y_data, x_test_fin_588)
oof_train_rf_f4, oof_test_rf_f4 = get_oof(rf_f4, x_train_fin_583, y_data, x_test_fin_583)
oof_train_rf_f5, oof_test_rf_f5 = get_oof(rf_f5, x_train_fin_560, y_data, x_test_fin_560)
oof_train_rf_f6, oof_test_rf_f6 = get_oof(rf_f6, x_train_fin_519, y_data, x_test_fin_519)
oof_train_rf_f7, oof_test_rf_f7 = get_oof(rf_f7, x_train_fin_518, y_data, x_test_fin_518)
oof_train_rf_f8, oof_test_rf_f8 = get_oof(rf_f8, x_train_fin_467, y_data, x_test_fin_467)

c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning 

In [34]:
oof_train_rf_m1, oof_test_rf_m1 = get_oof(rf_m1, x_train_main_738, y_data, x_test_main_738)
oof_train_rf_m2, oof_test_rf_m2 = get_oof(rf_m2, x_train_main_744, y_data, x_test_main_744)
oof_train_rf_m3, oof_test_rf_m3 = get_oof(rf_m3, x_train_main_728, y_data, x_test_main_728)
oof_train_rf_m4, oof_test_rf_m4 = get_oof(rf_m4, x_train_main_700, y_data, x_test_main_700)
oof_train_rf_m5, oof_test_rf_m5 = get_oof(rf_m5, x_train_main_657, y_data, x_test_main_657)
oof_train_rf_m6, oof_test_rf_m6 = get_oof(rf_m6, x_train_main_654, y_data, x_test_main_654)
oof_train_rf_m7, oof_test_rf_m7 = get_oof(rf_m7, x_train_main_636, y_data, x_test_main_636)
oof_train_rf_m8, oof_test_rf_m8 = get_oof(rf_m8, x_train_main_482, y_data, x_test_main_482)

c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning 

In [35]:
oof_train_rf_f, oof_test_rf_f = get_oof(rf_f, x_train_fin_519, y_data, x_test_fin_519)
oof_train_rf_m, oof_test_rf_m = get_oof(rf_m, x_train_main_482, y_data, x_test_main_482)

c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning 

In [36]:
oof_train_et_f, oof_test_et_f = get_oof(et_f, x_train_fin_519, y_data, x_test_fin_519)
oof_train_et_m, oof_test_et_m = get_oof(et_m, x_train_main_482, y_data, x_test_main_482)

c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning 

In [37]:
oof_train_gb_f, oof_test_gb_f = get_oof(gb_f, x_train_fin_519, y_data, x_test_fin_519)
oof_train_gb_m, oof_test_gb_m = get_oof(gb_m, x_train_main_482, y_data, x_test_main_482)

c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning 

In [38]:
oof_train_xgb_f, oof_test_xgb_f = get_oof(xgb_f, x_train_fin_519, y_data, x_test_fin_519)
oof_train_xgb_m, oof_test_xgb_m = get_oof(xgb_m, x_train_main_482, y_data, x_test_main_482)

c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning 

In [39]:
oof_train_llr, oof_test_llr = get_oof(llr, x_train_296, y_data, x_test_296)
oof_train_smlr, oof_test_smlr = get_oof(smlr, x_train_296, y_data, x_test_296)

c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python\venv\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning 

In [40]:
# oof_train_rf_svm = get_oof(svm, x_train_296, y_data)

In [315]:
x_train = np.concatenate((oof_train_rf_f1, oof_train_rf_f2, oof_train_rf_f3, oof_train_rf_f4, oof_train_rf_f5, oof_train_rf_f6, oof_train_rf_f7, oof_train_rf_f8, oof_train_rf_m1, oof_train_rf_m2, oof_train_rf_m3, oof_train_rf_m4, oof_train_rf_m5, oof_train_rf_m6, oof_train_rf_m7, oof_train_rf_m8, oof_train_rf_f, oof_train_rf_m, oof_train_et_f, oof_train_et_m, oof_train_gb_f, oof_train_gb_m, oof_train_xgb_f, oof_train_xgb_m, oof_train_llr, oof_train_smlr), axis=1)
x_train = pd.DataFrame(x_train)

x_test = np.concatenate((oof_test_rf_f1, oof_test_rf_f2, oof_test_rf_f3, oof_test_rf_f4, oof_test_rf_f5, oof_test_rf_f6, oof_test_rf_f7, oof_test_rf_f8, oof_test_rf_m1, oof_test_rf_m2, oof_test_rf_m3, oof_test_rf_m4, oof_test_rf_m5, oof_test_rf_m6, oof_test_rf_m7, oof_test_rf_m8, oof_test_rf_f, oof_test_rf_m, oof_test_et_f, oof_test_et_m, oof_test_gb_f, oof_test_gb_m, oof_test_xgb_f, oof_test_xgb_m, oof_test_llr, oof_test_smlr), axis=1)
x_test = pd.DataFrame(x_test)

In [316]:
x_train.columns = ['rf_f1', 'rf_f2', 'rf_f3', 'rf_f4', 'rf_f5', 'rf_f6', 'rf_f7', 'rf_f8', 'rf_m1', 'rf_m2', 'rf_m3', 'rf_m4', 'rf_m5', 'rf_m6', 'rf_m7', 'rf_m8', 'rf_f', 'rf_m', 'et_f', 'et_m', 'gb_f', 'gb_m', 'xgb_f', 'xgb_m', 'llr', 'smlr']
x_test.columns = ['rf_f1', 'rf_f2', 'rf_f3', 'rf_f4', 'rf_f5', 'rf_f6', 'rf_f7', 'rf_f8', 'rf_m1', 'rf_m2', 'rf_m3', 'rf_m4', 'rf_m5', 'rf_m6', 'rf_m7', 'rf_m8', 'rf_f', 'rf_m', 'et_f', 'et_m', 'gb_f', 'gb_m', 'xgb_f', 'xgb_m', 'llr', 'smlr']

## LAYER 2

In [317]:
le = LabelEncoder()
le.fit(x_train.rf_f1)

LabelEncoder()

In [318]:
for v in x_train.columns:
    x_train[v] = le.transform(x_train[v])
    x_test[v] = le.transform(x_test[v])

### rf

In [331]:
rf_t = RandomForestClassifier(n_estimators=175, random_state=3, max_depth=5, n_jobs=-1)

In [332]:
rf_t.fit(x_train, y_data)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=175, n_jobs=-1,
            oob_score=False, random_state=3, verbose=0, warm_start=False)

In [339]:
acc_id = x_test_load_296.acc_id.values

In [340]:
result = rf_t.predict(x_test)

In [344]:
finish = pd.DataFrame({'acc_id' : acc_id, 'result' : result})

In [348]:
finish.index = finish.acc_id

In [350]:
finish = finish.drop(['acc_id'], axis=1)

In [352]:
finish.to_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\result\\finish.csv')